<a href="https://colab.research.google.com/github/BruceAuHK/Dashboard_FFO/blob/master/Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import urllib.request
import json
import ssl
import plotly as py
import plotly.graph_objects as go
import ipywidgets as widgets
import numpy as np
from scipy import special 
import pandas as pd
from pandas.io.json import json_normalize
import plotly.express as px

py.offline.init_notebook_mode(connected=True)
context = ssl._create_unverified_context()


#To plot grapgh on colab run configure_plotly_browser_state() on each cell
configure_plotly_browser_state()
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

def get_data_from_HKMA(url):
    # Balance sheet
    url = url

    # retrieve data from hkma api, 100 entries per call, untill every data are retrieveed
    records = []
    offset = 0
    while True:
        with urllib.request.urlopen(url+"&offset="+str(offset), context=context) as req:
            data = json.loads(req.read().decode())
            records += data['result']['records']
        if data['result']['datasize'] <= 0:
            break
        else:
            offset += 100

    # now data retrieved from hkma are stored in 'records' in a chronological order
    return records


print("dependencies ready")


dependencies ready


# **Monetary Base**

In [ ]:
MBC = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/monetary-operation/monetary-base-endperiod?choose=end_of_month&from=1998-09&to=now&sortby=end_of_month&sortorder=asc'

#1. Size of the Monetary Base was published since 25 November 1998.
mbc_df = pd.json_normalize(get_data_from_HKMA(MBC))

#Convert dataframe to dataframe long format
mbc_dfl = pd.melt(mbc_df, id_vars = ['end_of_month'], value_vars =['cert_of_indebt', 'gov_notes_coins_circulation' , 'aggr_balance_bf_disc_win' , 'outstanding_efbn' , 'ow_lb_bf_disc_win' , 'mb_bf_disc_win_total'],var_name='Size of the Monetary Base', value_name='HK$ million' )

fig = px.area(mbc_dfl, x='end_of_month' , y='HK$ million', color = 'Size of the Monetary Base')

#To plot grapgh on colab run configure_plotly_browser_state() on each cell
configure_plotly_browser_state()
fig.show()


# 1. Money Multiplier = M3 / MB (HKD)

In [ ]:
# ---------------- 1. Money Multiplier = M3 / MB (HKD) ----------------

MSA = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/money/supply-adjusted?choose=end_of_month&from=1998-09&to=now&sortby=end_of_month&sortorder=asc'

#1. There is a break in data series due to the inclusion of short-term Exchange Fund placements of less than one month since April 1997.
msa_df = pd.json_normalize(get_data_from_HKMA(MSA))


In [ ]:
m3_hkd = msa_df[['end_of_month','m3_total']]

mb_tot = mbc_df[['end_of_month','mb_bf_disc_win_total']]

m3_hkd_Over_mb_tot = pd.merge(m3_hkd, mb_tot, on='end_of_month')

m3_hkd_Over_mb_tot['m3Overmb'] = (m3_hkd_Over_mb_tot['m3_total'] / m3_hkd_Over_mb_tot['mb_bf_disc_win_total'])


#To plot grapgh on colab run configure_plotly_browser_state() on each cell
configure_plotly_browser_state()
fig = px.line(m3_hkd_Over_mb_tot, x="end_of_month", y = "m3Overmb",
              
              labels={
                     "x": "End_of_month",
                     "y": "Sepal Width (cm)"
                 },
              title='m3_hkd_Over_mb_tot')
fig.show()



In [ ]:
#To plot grapgh on colab run configure_plotly_browser_state() on each cell
configure_plotly_browser_state()
fig = px.line(m3_hkd_Over_mb_tot, x="end_of_month", y = "m3_total",
              
              labels={
                     "x": "End_of_month",
                     "y": "m3_total"
                 },
              title='m3_total')
fig.show()

In [ ]:
#To plot grapgh on colab run configure_plotly_browser_state() on each cell
configure_plotly_browser_state()
fig = px.line(m3_hkd_Over_mb_tot, x="end_of_month", y = "mb_bf_disc_win_total",
              
              labels={
                     "x": "End_of_month",
                     "y": "mb_bf_disc_win_total"
                 },
              title='mb_bf_disc_win_total')
fig.show()

# 2. Total deposits & M3 (HKD)

In [ ]:
TD = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/customer-deposits-by-currency?choose=end_of_month&from=1998-09&to=no&sortby=end_of_month&sortorder=asc'

td_records_df = pd.json_normalize(get_data_from_HKMA(TD))

deposits_all = td_records_df[['end_of_month','deposits_all']]

m3_hkd_and_deposits_all  = pd.merge(m3_hkd, deposits_all, on='end_of_month')
#Convert dataframe to dataframe long format
m3_hkd_and_deposits_all_dfl = pd.melt(m3_hkd_and_deposits_all, id_vars = ['end_of_month'], value_vars =['m3_total', 'deposits_all'],var_name='var', value_name='HK$ million' )

fig = px.area(m3_hkd_and_deposits_all_dfl, x='end_of_month' , y='HK$ million', color = 'var')
#To plot grapgh on colab run configure_plotly_browser_state() on each cell
configure_plotly_browser_state()
fig.show()

# 3. Loan to Deposits


In [ ]:
BS = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/balance-sheet-ais?choose=end_of_month&from=1998-09&to=no&sortby=end_of_month&sortorder=asc'

bs_records_df = pd.json_normalize(get_data_from_HKMA(BS))
bs_records_df.head(10)





,end_of_month,liab_amtaisinhk_hkd,liab_amtaisinhk_fc,liab_amtaisinhk_total,liab_amtbanksabroad_hkd,liab_amtbanksabroad_fc,liab_amtbanksabroad_total,liab_depcust_hkd,liab_depcust_fc,liab_depcust_total,liab_ncdepoutst_hkd,liab_ncdepoutst_fc,liab_ncdepoutst_total,liab_othdebtinstoutst_hkd,liab_othdebtinstoutst_fc,liab_othdebtinstoutst_total,liab_capreserv_hkd,liab_capreserv_fc,liab_capreserv_total,liab_total_hkd,liab_total_fc,liab_total,asst_notescoins_hkd,asst_notescoins_fc,asst_notescoins_total,asst_amtaisinhk_hkd,asst_amtaisinhk_fc,asst_amtaisinhk_total,asst_amtbanksabroad_hkd,asst_amtbanksabroad_fc,asst_amtbanksabroad_total,asst_loanadvtocust_hkd,asst_loanadvtocust_fc,asst_loanadvtocust_total,asst_ncdepheld_hkd,asst_ncdepheld_fc,asst_ncdepheld_total,asst_ncdepheld_lb_hkd,asst_ncdepheld_lb_fc,asst_ncdepheld_lb_total,asst_ncdepheld_rlb_hkd,asst_ncdepheld_rlb_fc,asst_ncdepheld_rlb_total,asst_ncdepheld_dtc_hkd,asst_ncdepheld_dtc_fc,asst_ncdepheld_dtc_total,asst_ncdepheld_bkouthk_hkd,asst_ncdepheld_bkouthk_fc,asst_ncdepheld_bkouthk_total,asst_negodebt_hkd,asst_negodebt_fc,asst_negodebt_total,asst_negodebt_accbill_hkd,asst_negodebt_accbill_fc,asst_negodebt_accbill_total,asst_negodebt_fltnote_hkd,asst_negodebt_fltnote_fc,asst_negodebt_fltnote_total,asst_negodebt_gov_hkd,asst_negodebt_gov_fc,asst_negodebt_gov_total,asst_negodebt_other_hkd,asst_negodebt_other_fc,asst_negodebt_other_total,asst_invinshare_hkd,asst_invinshare_fc,asst_invinshare_total,asst_intinland_hkd,asst_intinland_fc,asst_intinland_total,asst_other_hkd,asst_other_fc,asst_other_total,total_asst_hkd,total_asst_fc,total_asst_total,no_of_report_instit,no_of_local_branch
0,1998-09,404753.0,229575.0,634328.0,230836.0,2598713.0,2829550.0,1623579.0,1302339.0,2925917.0,165729.0,52080.0,217809.0,3048.0,60040.0,63088.0,367978.0,224631.0,592610.0,2795922.0,4467379.0,7263301.0,13544.0,1324.0,14868.0,400519.0,236427.0,636946.0,175698.0,1993807.0,2169506.0,1707659.0,1653127.0,3360786.0,105037.0,42829.0,147866.0,97743.0,18807.0,116550.0,3792.0,1180.0,4972.0,1109.0,15.0,1124.0,2393.0,22827.0,25219.0,193787.0,388102.0,581889.0,2352.0,70124.0,72476.0,24305.0,151539.0,175844.0,130386.0,93752.0,224138.0,36745.0,72687.0,109431.0,15977.0,1838.0,17814.0,51800.0,850.0,52650.0,100093.0,180882.0,280975.0,2764114.0,4499187.0,7263301.0,344,1586
1,1998-10,404946.0,214191.0,619138.0,231375.0,2709763.0,2941138.0,1656415.0,1336206.0,2992621.0,163458.0,51461.0,214919.0,3391.0,60388.0,63779.0,379694.0,215485.0,595180.0,2839279.0,4587495.0,7426774.0,10994.0,2129.0,13123.0,404035.0,216111.0,620146.0,170393.0,2056508.0,2226901.0,1700338.0,1769714.0,3470052.0,104190.0,40985.0,145175.0,97351.0,18786.0,116138.0,3150.0,1142.0,4292.0,1109.0,15.0,1124.0,2580.0,21041.0,23620.0,199096.0,360440.0,559535.0,2057.0,62930.0,64987.0,23867.0,138171.0,162038.0,135107.0,86493.0,221601.0,38065.0,72845.0,110910.0,15882.0,1831.0,17714.0,51727.0,839.0,52566.0,125101.0,196459.0,321560.0,2781757.0,4645016.0,7426774.0,343,1588
2,1998-11,411508.0,214037.0,625546.0,231561.0,2673433.0,2904995.0,1661895.0,1316721.0,2978616.0,165051.0,49986.0,215037.0,3340.0,59490.0,62830.0,375823.0,213387.0,589209.0,2849178.0,4527055.0,7376233.0,10387.0,1837.0,12224.0,409861.0,221846.0,631707.0,172476.0,2166979.0,2339455.0,1701589.0,1626934.0,3328523.0,106028.0,40579.0,146608.0,99032.0,18888.0,117919.0,3080.0,1097.0,4177.0,1169.0,15.0,1184.0,2748.0,20580.0,23327.0,198648.0,351227.0,549876.0,1917.0,57567.0,59484.0,24348.0,127331.0,151679.0,134919.0,92676.0,227594.0,37465.0,73653.0,111118.0,15896.0,1796.0,17692.0,56741.0,836.0,57577.0,112166.0,180406.0,292572.0,2783792.0,4592441.0,7376233.0,340,1586
3,1998-12,414367.0,207838.0,622205.0,231607.0,2571866.0,2803473.0,1670437.0,1329590.0,3000028.0,163491.0,45863.0,209354.0,3423.0,59487.0,62910.0,361571.0,194935.0,556505.0,2844895.0,4409580.0,7254475.0,11321.0,2205.0,13526.0,411169.0,218075.0,629243.0,166805.0,2145598.0,2312403.0,1695027.0,1609400.0,3304427.0,106189.0,38290.0,144480.0,99382.0,17888.0,11727

In [ ]:

bs_records_df['LtoD_hkd'] = (bs_records_df['asst_loanadvtocust_hkd'] / bs_records_df['liab_depcust_hkd'])

#To plot grapgh on colab run configure_plotly_browser_state() on each cell
configure_plotly_browser_state()


fig = px.line(bs_records_df, x="end_of_month", y = "LtoD_hkd",
              title='LtoD_hkd')
fig.show()




In [ ]:
#Convert dataframe to dataframe long format
bs_records_dfl = pd.melt(bs_records_df, id_vars = ['end_of_month'], value_vars =['asst_loanadvtocust_hkd', 'liab_depcust_hkd'],var_name='var', value_name='HK$ million' )

fig = px.line(bs_records_dfl, x='end_of_month' , y='HK$ million', color = 'var')
#To plot grapgh on colab run configure_plotly_browser_state() on each cell
configure_plotly_browser_state()
fig.show()

## 3.2. Loan to Deposits (FC)


In [ ]:

bs_records_df['LtoD_fc'] = (bs_records_df['asst_loanadvtocust_fc'] / bs_records_df['liab_depcust_fc'])

#To plot grapgh on colab run configure_plotly_browser_state() on each cell
configure_plotly_browser_state()


fig = px.line(bs_records_df, x="end_of_month", y = "LtoD_hkd",
              title='LtoD_hkd')
fig.show()




In [ ]:
#Convert dataframe to dataframe long format
bs_records_dfl = pd.melt(bs_records_df, id_vars = ['end_of_month'], value_vars =['asst_loanadvtocust_fc', 'liab_depcust_fc'],var_name='var', value_name='HK$ million' )

fig = px.line(bs_records_dfl, x='end_of_month' , y='HK$ million', color = 'var')
#To plot grapgh on colab run configure_plotly_browser_state() on each cell
configure_plotly_browser_state()
fig.show()

## 3.3 Loan to Deposits (Total)

In [ ]:

bs_records_df['LtoD_total'] = (bs_records_df['asst_loanadvtocust_total'] / bs_records_df['liab_depcust_total'])

#To plot grapgh on colab run configure_plotly_browser_state() on each cell
configure_plotly_browser_state()


fig = px.line(bs_records_df, x="end_of_month", y = "LtoD_hkd",
              title='LtoD_hkd')
fig.show()




In [ ]:
#Convert dataframe to dataframe long format
bs_records_dfl = pd.melt(bs_records_df, id_vars = ['end_of_month'], value_vars =['asst_loanadvtocust_total', 'liab_depcust_total'],var_name='var', value_name='HK$ million' )

fig = px.line(bs_records_dfl, x='end_of_month' , y='HK$ million', color = 'var')
#To plot grapgh on colab run configure_plotly_browser_state() on each cell
configure_plotly_browser_state()
fig.show()